In [1]:
from langame_client import LangameClient
from pprint import pprint
from firebase_admin import firestore
import requests
from bs4 import BeautifulSoup
import json
import openai
c = LangameClient()

In [2]:
URL = "https://conversationstartersworld.com/philosophical-questions/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
philosophical_questions = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]
memes = []
for e in c._firestore_client.collection("memes").stream():
    memes.append(e.get("content"))
memes = memes + [" ".join(e.split(" ")[1:]) for e in philosophical_questions[3:-5]]
URL = "https://conversationstartersworld.com/deep-conversation-topics/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
deep = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]
memes += [" ".join(e.split(" ")[1:]) for e in deep[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)

URL = "https://conversationstartersworld.com/hypothetical-questions/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
hypothetical = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]
memes += [" ".join(e.split(" ")[1:]) for e in hypothetical[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)
URL = "https://conversationstartersworld.com/open-ended-questions/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
open_ended = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]
memes += [" ".join(e.split(" ")[1:]) for e in open_ended[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)

URL = "https://conversationstartersworld.com/closed-ended-questions-3/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
closed_ended = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]
memes += [" ".join(e.split(" ")[1:]) for e in closed_ended[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)
URL = "https://conversationstartersworld.com/good-questions-to-ask/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
good_question_to_ask = [e.text for e in soup.find(id="genesis-content").find("article").find("div").find_all("p")]
memes += [" ".join(e.split(" ")[1:]) for e in good_question_to_ask[3:-5]]

with open('memes.json', 'w') as outfile:
    json.dump({"memes": memes}, outfile, indent=4)

In [6]:
file_name = 'data/generate-generalist-0.0.4.jsonl'

In [3]:
with open(file_name, '') as outfile:
    for meme in memes:
        json.dump({
            "prompt": "",
            "completion": f" {meme}\n###\n"
        }, outfile)
        outfile.write('\n')

In [6]:
f = openai.File.create(
  file=open(file_name),
  purpose="fine-tune"
)
f_t = openai.FineTune.create(
    training_file=f["id"],
    model="curie"
)

Upload progress: 100%|██████████| 184k/184k [00:00<00:00, 235kit/s]


In [11]:
model = c._openai_client.get_fine_tune(file_name)
model

<FineTune fine-tune id=ft-yUmFBzIjdR9yvflLyhlUNNDW at 0x13f0eb4a0> JSON: {
  "created_at": 1630082058,
  "fine_tuned_model": "curie:ft-louis-personal-2021-08-27-16-57-14",
  "hyperparams": {
    "batch_size": 4,
    "learning_rate_multiplier": 0.1,
    "n_epochs": 4,
    "prompt_loss_weight": 0.1,
    "use_packing": null
  },
  "id": "ft-yUmFBzIjdR9yvflLyhlUNNDW",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-KwcHNgfGe4pqdKDLQIJt99UZ",
  "result_files": [
    {
      "bytes": 83555,
      "created_at": 1630083439,
      "filename": "compiled_results.csv",
      "id": "file-bZ4mWVMHUmTvo75y14xTrGOb",
      "model": null,
      "object": "file",
      "purpose": "fine-tune-results",
      "status": "processed",
      "status_details": null
    }
  ],
  "status": "succeeded",
  "training_files": [
    {
      "bytes": 184071,
      "created_at": 1630082057,
      "filename": "data/generate-generalist-0.0.4.jsonl",
      "id": "file-7pxSr5bcoFN8W3UGfVTJbxit",
     

In [13]:
p = {}
p["type"] = file_name
p["template"] = "[TOPIC]:"
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": model["fine_tuned_model"],
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\n###\n"],
        }
    }
}
p_col = c._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 8, 27, 17, 32, 0, 932031, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x13f6e5970>)


In [34]:
openai.Completion.create(
  model=model["fine_tuned_model"],
  prompt="
  max_tokens=300,
)
  stop=["\n###\n"]
)

<OpenAIObject text_completion id=cmpl-3bb9PbAkCra9NSEJrOTKDhDxi4yCP at 0x13f15c770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "While most of humanity is focused on avoiding harm, a small but very vocal minority has decided that the best way to avoid harm is to actively seeks it out. Called \u201cthe optimists\u201d by the media, this group includes anarchists, Daredevil bloggers, thrill-seekers, and other \u2013 sometimes bizarre \u2013 personalities."
    }
  ],
  "created": 1630089955,
  "id": "cmpl-3bb9PbAkCra9NSEJrOTKDhDxi4yCP",
  "model": "curie:ft-louis-personal-2021-08-27-16-57-14",
  "object": "text_completion"
}